# Simulate RHDO Data for In-Silico LOD

* The aim of this notebook is to test what kind of depths and fetal fractions the method works at.

* Pretends we have a list of filtered snps 

In [187]:
import pandas as pd
from random import choices
from collections import Counter
import numpy as np
import seaborn as sns
min_snps_per_block = 25

In [188]:
number_of_snps = 1000

def get_fetal_fraction_x_linked(cHapA, cHapB):
    """
    Use the formula from the Birmingham paper to calculate the fetal \
    fraction for X-linked data.
    
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4864947/
    
    cffDNA % = [(|cHapA – cHapB| * 2) / (cHapA + cHapB + |cHapA – cHapB|)] * 100
    
    """
    
    fetal_fraction = (abs(cHapA-cHapB)*2) / (cHapA + cHapB + abs(cHapA-cHapB)) *100
    
    return fetal_fraction

In [189]:
snp_list = []

depths = [1000]
ffs = [15]
n_samples = 10

for depth in depths:
    
    for ff in ffs:
        
        for n in range(n_samples):
            
            for snp in range(number_of_snps):
            
                # sample with a specific weight 
                bias = (0.5 + ff/100) / (1+(ff/100))
                
                random_depth = np.random.normal(depth,depth//3, size=1).astype(int)[0]
                
                if random_depth < 20:
                    
                    random_depth =20
            
                cnt = Counter(choices(['C', 'T'], k=random_depth, weights=[1-bias, bias]))
                hapa = cnt['T']
                hapb = cnt['C']
                calc_ff = get_fetal_fraction_x_linked(hapa, hapb)/100
                
                if 
            
                my_snp = ['X', snp , 'C', 'T', depth, ff, hapa, hapb, n]
                snp_list.append(my_snp)
                
            

In [190]:
df = pd.DataFrame(snp_list, columns=['CHROM', 'POS', 'REF', 'ALT', 'DEPTH', 'FF', 'hapA_count', 'hapB_count', 'sample'])

In [191]:
df.head()

,CHROM,POS,REF,ALT,DEPTH,FF,hapA_count,hapB_count,sample
0,X,0,C,T,1000,15,614,451,0
1,X,1,C,T,1000,15,447,321,0
2,X,2,C,T,1000,15,887,612,0
3,X,3,C,T,1000,15,500,398,0
4,X,4,C,T,1000,15,708,590,0


In [192]:
def sprt_calculate_d(fetal_fraction):
    """
    Calculate the d value needed for SPRT analysis:
    
    See the following papers for more information:
    
    1) https://www.ncbi.nlm.nih.gov/pubmed/21148127
    
    2) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1934923/bin/pnas_0705765104_index.html
    
    The d value is used by the function sprt_perform_test()
    
    """
    
    # q0 is the allele imbalance if the null hypothesis is correct i.e. the fetus has the normal allele.
    # the 0.5 value is because we would expect to see this much deviation from the 0.5 allele balance seen \
    # in HET SNPs and then some more deviancy depending on the fetal fraction.
    
    
    q0 = 0.5 - (fetal_fraction / 2)
        
    # q1 is the allele imbalance if the null hypothesis is incorrect i.e. the fetus has the mutant allele.

    q1 = 0.5 + (fetal_fraction / 2)
        
    d_value = (1 - q1) / (1 - q0)
    
    return d_value

def sprt_calculate_g(fetal_fraction):
    """
    Calculate the g value needed for SPRT analysis:
    
    See the following papers for more information:
    
    1) https://www.ncbi.nlm.nih.gov/pubmed/21148127
    
    2) https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1934923/bin/pnas_0705765104_index.html
    
    The g value is used by the function sprt_perform_test()
    
    """
    
    # q0 is the allele imbalance if the null hypothesis is correct i.e. the fetus has the normal allele.
    # the 0.5 value is because we would expect to see this much deviation from the 0.5 allele balance seen \
    # in HET SNPs and then some more deviancy depending on the fetal fraction.
    
    q0 = 0.5 - (fetal_fraction/2)

    # q1 is the allele imbalance if the null hypothesis is incorrect i.e. the fetus has the mutant allele.

    q1 = 0.5 + (fetal_fraction/2)

    g_value = ( q1*(1 - q0) ) / ( q0*(1 - q1) )
    
    return g_value

def sprt_calculate_upper_boundry(total_read_count, d_value, g_value):
    """
    Calculate the upper value for the SPRT classification.
    
    """
    
    return (( np.log(1200) / total_read_count ) - np.log(d_value)) / np.log(g_value)

def sprt_calculate_lower_boundry(total_read_count, d_value, g_value):
    """
    Calculate the lower value for the SPRT classification.
    
    """
    
    return (( np.log(float(1) / float(1200) )/ total_read_count ) - np.log(d_value)) / np.log(g_value)

def sprt_perform_test(df, d_value, g_value):
    """
    Function to call the status of each snp as hapA, hapB or Unclassified.
    
    Input:
    
    df = The processed dataframe containing the X-linked SNP data.
    d_value = The value calculated by sprt_calculate_d()
    g_value = The value calculated by sprt_calculate_g()
    
    Output:
    
    df = A new dataframe containing seven additional rows:
    
    cumulative_sum_hapa
    cumulative_sum_hapb
    cumulative_ratio
    cumulative_total
    upper_boundry
    lower_boundry
    status
    
    
    """
    
    cumulative_sum_hapa = 0
    cumulative_sum_hapb = 0
    snp_count = 0
    
    # Loop through each snp
    for snp in df.itertuples():
        
        # Calculate the cumlative sum
        cumulative_sum_hapa = cumulative_sum_hapa + snp.hapA_count
        cumulative_sum_hapb = cumulative_sum_hapb + snp.hapB_count
        
        # Calculate the cumlative allele balance between hapA and hapB
        cumulative_ratio = cumulative_sum_hapa / (cumulative_sum_hapa + cumulative_sum_hapb)
        
        cumulative_total = cumulative_sum_hapa + cumulative_sum_hapb
        
        # Calculate upper and lower SPRT boundaries
        upper_boundry = sprt_calculate_upper_boundry(cumulative_total, d_value, g_value )
        
        lower_boundry = sprt_calculate_lower_boundry(cumulative_total, d_value, g_value)
        
        # call blocks either hapA, hapB or Unclassified
        
        if (cumulative_ratio > upper_boundry) and snp_count >= min_snps_per_block:
            
            status = 'hapA'
            cumulative_sum_hapa = 0
            cumulative_sum_hapb = 0
            snp_count = 0
            
            
        elif cumulative_ratio < lower_boundry and snp_count >= min_snps_per_block:
            
            status = 'hapB'
            cumulative_sum_hapa = 0
            cumulative_sum_hapb = 0
            snp_count = 0
            
        else:
            
            status = 'Unclassified'
            
        snp_count = snp_count + 1
        
        #update dataframe with calculated values
        df.at[snp.Index, 'cumulative_sum_hapa'] = cumulative_sum_hapa
        df.at[snp.Index, 'cumulative_sum_hapb'] = cumulative_sum_hapb
        df.at[snp.Index, 'cumulative_ratio'] = cumulative_ratio
        df.at[snp.Index, 'cumulative_total'] = cumulative_total
        df.at[snp.Index, 'upper_boundry'] = upper_boundry
        df.at[snp.Index, 'lower_boundry'] = lower_boundry
        df.at[snp.Index, 'status'] = status
        
    return df

def call_blocks(df):
    
    block_df = []
    
    prev_status = 'Unclassified'
    snp_count = 0
    block_id = 0
    first = True
    
    for snp in df.itertuples():
        
        if first == True:
            
            prev_pos = snp.POS
            first = False
        
        status = snp.status
        
        if status != prev_status:
            
            block_df.append([block_id, prev_pos, snp.POS, snp_count, status])
            
            prev_pos = snp.POS
            snp_count = 0
            block_id = block_id + 1
        
            
        snp_count = snp_count + 1
        
    return block_df

In [193]:
results = []

for depth in depths:
    
    for ff in ffs:
        
        for n in range(n_samples):
        
            grouped = df[(df['DEPTH'] == depth) & ( df['FF'] == ff) & (df['sample'] ==n)]

            hapa = grouped['hapA_count'].sum()
            hapb = grouped['hapB_count'].sum()
            fetal_fraction = get_fetal_fraction_x_linked(hapa, hapb)

            d_value = sprt_calculate_d(fetal_fraction / 100)
            g_value = sprt_calculate_g(fetal_fraction / 100)

            df_fwd = sprt_perform_test(grouped, d_value, g_value)

            df_rev = sprt_perform_test(grouped.sort_values(by='POS', ascending=False), d_value, g_value)

            blocks_fwd = call_blocks(df_fwd)
            blocks_rev = call_blocks(df_rev)

            mean_snps_per_block = np.mean([x[3] for x in blocks_fwd] + [x[3] for x in blocks_rev])
            hapas = [x[4] for x in blocks_fwd] + [x[4] for x in blocks_rev]

            count = Counter(hapas)

            hapa_count = 0
            hapb_count = 0
            unclassed_count = 0

            if 'hapA' in count:

                hapa_count = count['hapA']

            if 'hapB' in count:

                hapa_count = count['hapB']

            if 'Unclassified' in count:

                unclassed_count = count['Unclassified']

            results.append([depth, ff, n, mean_snps_per_block, hapa_count, hapb_count, unclassed_count])
        
        

In [194]:
results_df = pd.DataFrame(results, columns = ['depth', 'ff', 'n', 'mean_snps_per_block', 'hapa_count', 'hapb_count', 'unclassed'])